<a href="https://colab.research.google.com/github/alihuss1017/LSTM-Weather-Prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
gh_token = userdata.get('GITHUB_TOKEN')

In [2]:
!git clone https://{gh_token}@github.com/alihuss1017/LSTM-Weather-Prediction.git

Cloning into 'LSTM-Weather-Prediction'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 12.89 KiB | 249.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
cd LSTM-Weather-Prediction

/content/LSTM-Weather-Prediction


In [42]:
import pandas as pd

df = pd.read_csv('data/seattle-weather.csv')

In [43]:
print(f'''Number of null values:\n{df.isnull().sum()}\n\nNumber of duplicated rows: {df.duplicated().sum()}''')

Number of null values:
date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

Number of duplicated rows: 0


In [44]:
df = df.set_index(df["date"])
df = df.drop('date', axis = 1)


In [153]:
mu, std = df['temp_max'].mean(), df['temp_max'].std()

In [154]:
df_encoded = pd.get_dummies(df, columns = ['weather'])

In [155]:
df_encoded

,precipitation,temp_max,temp_min,wind,weather_drizzle,weather_fog,weather_rain,weather_snow,weather_sun
date,,,,,,,,,
2012-01-01,0.0,12.8,5.0,4.7,True,False,False,False,False
2012-01-02,10.9,10.6,2.8,4.5,False,False,True,False,False
2012-01-03,0.8,11.7,7.2,2.3,False,False,True,False,False
2012-01-04,20.3,12.2,5.6,4.7,False,False,True,False,False
2012-01-05,1.3,8.9,2.8,6.1,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
2015-12-27,8.6,4.4,1.7,2.9,False,False,True,False,False
2015-12-28,1.5,5.0,1.7,1.3,False,False,True,False,False
2015-12-29,0.0,7.2,0.6,2.6,False,True,False,False,False


In [125]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columns_to_normalize = df_encoded.select_dtypes(include='float').columns.tolist()

df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

In [128]:
import torch
from torch.utils.data import Dataset, DataLoader

class WeatherDataset(Dataset):

  def __init__(self, data_df, seq_len):
    self.data = data_df
    self.seq_len = seq_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    cols_to_cast = self.data.select_dtypes(include = ['object', 'bool']).columns.tolist()
    for col in cols_to_cast:
      self.data[col] = self.data[col].astype('int')

    x = torch.tensor(self.data.iloc[idx:idx+self.seq_len].values, dtype = torch.float32)
    y = torch.tensor(self.data['temp_max'].iloc[idx+self.seq_len+1], dtype = torch.float32)

    return x, y


In [150]:
from torch.utils.data import Subset

dataset = WeatherDataset(data_df = df_encoded, seq_len = 5)
train_len = int(0.7 * len(dataset))

train_data = Subset(dataset, range(train_len))
val_data = Subset(dataset, range(train_len, len(dataset)))


train_loader = DataLoader(train_data, batch_size = 32, num_workers = 2, drop_last = True)
val_loader = DataLoader(val_data, batch_size = 32, num_workers = 2, drop_last = True )